<center><h1>UK Accidents
<h1>Accident Severity Predictive Model
<h2>Eugenio Zuccarelli</center>
    
---
   
Develop a predictive model able to predict whether a driver is likely to have a severe or fatal accident given biographical information and vehicle characteristics.

The probability in output of the predictive model can be used as a "Drive Score" highlighting the overall riskiness of a driver.
    
---

## 1. Libraries

In [6]:
import interpretableai.iai as ai
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.cluster.hierarchy as sh
import sklearn.cluster as sc
import sklearn.ensemble as se
import sklearn.linear_model as lm
import sklearn.model_selection as ms
import sklearn.metrics as sm
import xgboost as xg
from IPython.display import display
from julia import Distributed


# Enable Parallel Processing
Distributed.addprocs(23)

# Jupyter Plots
%matplotlib inline

## 2. Parameters

In [7]:
PREDICTIVE_FEATURES = ['Vehicle_Type','Towing_and_Articulation','Was_Vehicle_Left_Hand_Drive?', 
                       'Sex_of_Driver', 'Age_of_Driver', 'Engine_Capacity_(CC)', 'Propulsion_Code',
                       'Age_of_Vehicle', 'Driver_IMD_Decile', 'Driver_Home_Area_Type', 'Vehicle_IMD_Decile']

DESCRIPTIVE_FEATURES = ['Location_Easting_OSGR', 'Location_Northing_OSGR', 'Longitude', 
                        'Latitude', 'Police_Force',
                        'Number_of_Vehicles', 'Number_of_Casualties', 'Date', 
                        'Day_of_Week', 'Time', 'Local_Authority_(District)', 
                        'Local_Authority_(Highway)', '1st_Road_Class', '1st_Road_Number', 
                        'Road_Type', 'Speed_limit', 'Junction_Detail', 
                        'Junction_Control', '2nd_Road_Class', '2nd_Road_Number', 
                        'Pedestrian_Crossing-Human_Control', 'Carriageway_Hazards', 
                        'Pedestrian_Crossing-Physical_Facilities', 'Light_Conditions', 'Weather_Conditions', 
                        'Road_Surface_Conditions', 'Special_Conditions_at_Site', 'Urban_or_Rural_Area', 
                        'Did_Police_Officer_Attend_Scene_of_Accident', 'LSOA_of_Accident_Location',
                        'Vehicle_Manoeuvre','Vehicle_Location-Restricted_Lane', 'Junction_Location',
                        'Skidding_and_Overturning', 'Hit_Object_in_Carriageway','Vehicle_Leaving_Carriageway', 
                        'Hit_Object_off_Carriageway','1st_Point_of_Impact','Journey_Purpose_of_Driver',
                        'Pedestrian_Location', 'Pedestrian_Movement', 'Pedestrian_Road_Maintenance_Worker', 'Casualty_Type',
                        'Casualty_Home_Area_Type', 'Casualty_IMD_Decile', 'Casualty_Severity']

CATEGORICAL_FEATURES = ['Vehicle_Type', 'Towing_and_Articulation', 'Sex_of_Driver', 
                        'Propulsion_Code', 'Driver_Home_Area_Type']
BINARY_FEATURES = ['Was_Vehicle_Left_Hand_Drive']

## 3. Data Loading

---

The datasets consist of:
1. An accident-level dataset. Each row contains information on the accident's location, time of the day, etc.
2. A casualty-level dataset with biographical information on the casualty.
3. A vehicle-level dataset listing the features of each vehicle involved in the accident.

Due to the nature of the datasets, we need to make sure that the features used in the predictive model do not contain ex-post information. Only after, we can ensure a useable model.

---


In [8]:
accidents = pd.read_csv('Data/Accidents.csv', low_memory=False)
casualties = pd.read_csv('Data/Casualties.csv', low_memory=False)
vehicles = pd.read_csv('Data/Vehicles.csv', low_memory=False)

In [9]:
# For each vehicle involved in an accident, select the highest severity (lowest score) experienced by a person
casualties_vehicles = casualties.groupby(['Accident_Index', 'Vehicle_Reference'])['Casualty_Severity'].min().reset_index()
data = vehicles.merge(casualties_vehicles, how='left', on=['Accident_Index', 'Vehicle_Reference'])

# Restrict the dataset to ex-ante features
columns = PREDICTIVE_FEATURES
columns.append('Casualty_Severity')

data = data[columns]

## 4. Data Preprocessing

### Aggregate Unknown and Missing

In [10]:
data['Sex_of_Driver'] = data['Sex_of_Driver'].replace(3, -1)

### Remove NAs

In [11]:
data['Casualty_Severity'].fillna(value = 0, inplace = True)

In [12]:
data = data.replace(-1, np.nan)

### Target Variable Encoding

In [13]:
def severity(x):
    if x in (0.0, 3.0):
        return 0
    else:
        return 1

In [14]:
data['Casualty_Severity'] = data['Casualty_Severity'].apply(severity)

### One-Hot Encoding for Categorical Variables

In [15]:
for c in CATEGORICAL_FEATURES:
    if c in data.columns:
        one_hot = pd.get_dummies(data[c], prefix=c)
        data = data.drop(c, axis=1)
        data = data.join(one_hot)

### Binary Variables Encoding

In [16]:
# Convert 1 and 2 to False and True
for b in BINARY_FEATURES:
    if b in data.columns:
        data[b] = data[b].map({1: 0, 2: 1})
        data[b] = data[b].astype(bool)

## 5. Split Train and Test Set

In [17]:
X = data[data.columns[~data.columns.isin(['Casualty_Severity'])]]
y = data['Casualty_Severity']

(train_X, train_y), (test_X, test_y) = ai.split_data('classification', X, y, seed=1)

In [18]:
data

,Was_Vehicle_Left_Hand_Drive?,Age_of_Driver,Engine_Capacity_(CC),Age_of_Vehicle,Driver_IMD_Decile,Vehicle_IMD_Decile,Casualty_Severity,Vehicle_Type_1.0,Vehicle_Type_2.0,Vehicle_Type_3.0,...,Propulsion_Code_2.0,Propulsion_Code_3.0,Propulsion_Code_5.0,Propulsion_Code_6.0,Propulsion_Code_7.0,Propulsion_Code_8.0,Propulsion_Code_12.0,Driver_Home_Area_Type_1.0,Driver_Home_Area_Type_2.0,Driver_Home_Area_Type_3.0
0,1.0,32.0,1995.0,5.0,8.0,8.0,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0
1,1.0,48.0,1798.0,6.0,1.0,1.0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
2,1.0,NaN,NaN,NaN,NaN,NaN,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1.0,40.0,1797.0,6.0,3.0,3.0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
4,1.0,21.0,NaN,NaN,5.0,5.0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226404,1.0,67.0,998.0,4.0,NaN,NaN,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
226405,1.0,24.0,3000.0,21.0,NaN,NaN,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
226406,1.0,NaN,2400.0,5.0,NaN,NaN,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
226407,1.0,60.0,49.0,10.0,NaN,NaN,1,0,1,0,...,0,0,0,0,0,0,0,0,0,1


## 6. Imputation

---

In this dataset, the missing values have been substituted wiith -1. Keeping such values throughout the dataset would introduce relationships across the data which are non-descriptive of the true nature of the data. Therefore, we proceed substituting -1 with NaN, and subsequently impute the dataset. To avoid biasing the imputation, we fit the imputation learner with the training set only, without the y variable, for then transforming training and test sets with such learner.

---

In [ ]:
lnr = ai.ImputationLearner(method='opt_knn', random_seed=1, cluster=True)

lnr.fit(train_X)

train_X = lnr.transform(train_X)
test_X = lnr.transform(test_X)

### Logistic Regression

In [ ]:
# Logistic Regression Model
model = lm.LogisticRegression()

# Hyperparameters
params = {
 'C': [0.001, 0.01, 0.1, 1, 10, 100],
 'penalty': ['l1', 'l2'],
 'max_iter': [1000],
 'class_weight': ['balanced']
}

# Grid Search
clf = ms.GridSearchCV(model, params)

# Model Fit
model = clf.fit(train_X, train_y)

### Logistic Regression Performance

In [ ]:
predictions_train = model.predict_proba(train_X)[:, 1]
predictions_test = model.predict_proba(test_X)[:, 1]
print('In Sample AUC:', sm.roc_auc_score(train_y, predictions_train))
print('Out of Sample AUC: ', sm.roc_auc_score(test_y, predictions_test))

### Random Forests

In [ ]:
# Logistic Regression Model
model = se.RandomForestClassifier()

# Hyperparameters
params = {
 'max_depth': [10],
 'n_estimators': [100, 1000],
 'class_weight': [{0: 1, 1: 4}]
}

# Grid Search
clf = ms.GridSearchCV(model, params)

# Model Fit
model = clf.fit(train_X, train_y)

### Random Forest Performance

In [ ]:
predictions_train = model.predict_proba(train_X)[:, 1]
predictions_test = model.predict_proba(test_X)[:, 1]
print('In Sample AUC:', sm.roc_auc_score(train_y, predictions_train))
print('Out of Sample AUC: ', sm.roc_auc_score(test_y, predictions_test))

### Gradient Boosting

In [ ]:
# Gradient Boosting Model
model = xg.XGBClassifier()

# Hyperparameters
params = {
        'nthread': [4],
        'eta': [0.001, 0.01],
        'max_depth': [10],
        'n_estimators': [50, 100],
        'scale_pos_weight': [14]
        }

# Grid Search
grid = ms.GridSearchCV(model, params)

# Model Fit
model = grid.fit(train_X, train_y)

### Gradient Boosting Performance

In [ ]:
predictions_train = model.predict_proba(train_X)[:, 1]
predictions_test = model.predict_proba(test_X)[:, 1]
print('In Sample AUC:', sm.roc_auc_score(train_y, predictions_train))
print('Out of Sample AUC: ', sm.roc_auc_score(test_y, predictions_test))

### Optimal Classification Trees

In [ ]:
# Model
grid = ai.GridSearch(
    ai.OptimalTreeClassifier(
        random_seed=1,
    ),
    max_depth=7,
    treat_unknown_level_missing=True,
    missingdatamode='separate_class',
    criterion='gini'
)

# Train the Model
grid.fit(train_X, train_y)

### Optimal Classification Trees Performance

In [ ]:
print(grid.score(train_X, train_y, criterion='auc'))
print(grid.score(test_X, test_y, criterion='auc'))